# Hyperparamter Tuning via Optuna

## Global Optuna

### Loading Libraries

In [1]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Warnings
import warnings

# Time
import time

# Notebook Optimizer
from tqdm.notebook import tqdm

# Scikit-Learn
from sklearn.metrics import f1_score
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# TensorFlow
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential  
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Dense, Dropout 

# Optuna
import optuna

In [2]:
warnings.filterwarnings("ignore")

In [3]:
from tqdm.notebook import tqdm
import time

for i in tqdm(range(10), desc="Loading"):
    time.sleep(0.1)

Loading:   0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
!pip install tensorflow

### Function Definition

#### Placing Model Architecture Function

In [5]:
# def create_model(trial: optuna.trial.Trial, input_size: int): 
#     model = Sequential()
#     model.add(Dense(input_size,input_shape=(input_size,),activation='relu')) 

#     num_layers = trial.suggest_int('num_layers',low=0,high=3)  
#     for layer_i in range(num_layers):  
#         n_units = trial.suggest_int(f'n_units_layer_{layer_i}',low=10,high=50,step=5)  
#         dropout_rate = trial.suggest_float(f'dropout_rate_layer_{layer_i}',low=0,high=0.5)  
#         actv_func = trial.suggest_categorical(f'actv_func_layer_{layer_i}',['relu','tanh','elu'])  

#         model.add(Dropout(dropout_rate))  
#         model.add(Dense(n_units,activation=actv_func)) 

#     model.add(Dense(1,activation='sigmoid')) 
#     return model 

# ---------------------------- OPTUNA: MODEL ----------------------------

def create_model(trial, input_size): 
    model = Sequential()
    model.add(Dense(input_size, input_shape=(input_size,), activation='relu')) 

    num_layers = trial.suggest_int('num_layers', 0, 3)
    for layer_i in range(num_layers):
        n_units = trial.suggest_int(f'n_units_layer_{layer_i}', 10, 50, step=5)
        dropout_rate = trial.suggest_float(f'dropout_rate_layer_{layer_i}', 0.0, 0.5)
        actv_func = trial.suggest_categorical(f'actv_func_layer_{layer_i}', ['relu', 'tanh', 'elu'])

        model.add(Dropout(dropout_rate))
        model.add(Dense(n_units, activation=actv_func))

    model.add(Dense(1, activation='sigmoid'))
    return model

#### Optimizers Function

In [6]:
# def create_optimizer(trial: optuna.trial.Trial): 
# 	opt_kwargs = {} 
# 	opt_selected = trial.suggest_categorical('optimizer', ['Adam','SGD']) 
# 	if opt_selected == 'SGD': 
# 		opt_kwargs['lr'] = trial.suggest_float('sgd_lr',1e-5,1e-1,log=True) 
# 		opt_kwargs['momentum'] = trial.suggest_float('sgd_momentum',1e-5,1e-1,log=True) 
# 	else: #’Adam’ 
# 		opt_kwargs['lr'] = trial.suggest_float('adam_lr',1e-5,1e-1,log=True) 

# 	optimizer = getattr(tf.optimizers,opt_selected)(**opt_kwargs) 
# 	return optimizer

# ---------------------------- OPTUNA: OPTIMIZER ----------------------------

def create_optimizer(trial):
    opt_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD'])
    if opt_name == 'SGD':
        return SGD(
            learning_rate=trial.suggest_float('sgd_lr', 1e-5, 1e-1, log=True),
            momentum=trial.suggest_float('sgd_momentum', 1e-5, 1e-1, log=True)
        )
    else:
        return Adam(learning_rate=trial.suggest_float('adam_lr', 1e-5, 1e-1, log=True))

#### Custom Metric

In [7]:
# def recall_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     recall = true_positives / (possible_positives + K.epsilon())
#     return recall

# def precision_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     return precision

# def f1_m(y_true, y_pred):
#     precision = precision_m(y_true, y_pred)
#     recall = recall_m(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))

# ---------------------------- PERSONALIZED METRICS ----------------------------

@tf.function
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

@tf.function
def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

@tf.function
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

#### Preprocessing Function

In [8]:
# def preprocessing(X: pd.DataFrame, 
#                   numeric_preprocessor, categorical_preprocessor,
#                   is_train = True
#                  ):
#     if is_train:
#         X[numerical_feats] = numeric_preprocessor.fit_transform(X[numerical_feats])
#         X_cat = categorical_preprocessor.fit_transform(X[categorical_feats]).toarray()
#         X_cat = pd.DataFrame(X_cat,columns=categorical_preprocessor.get_feature_names_out())
#         X = X.drop(columns=categorical_feats).reset_index(drop=True)
#         X = pd.concat([X,X_cat],axis=1)
#     else:
#         X[numerical_feats] = numeric_preprocessor.transform(X[numerical_feats])
#         X_cat = categorical_preprocessor.transform(X[categorical_feats]).toarray()
#         X_cat = pd.DataFrame(X_cat,columns=categorical_preprocessor.get_feature_names_out())
#         X = X.drop(columns=categorical_feats).reset_index(drop=True)
#         X = pd.concat([X,X_cat],axis=1)
    
#     return X, numeric_preprocessor, categorical_preprocessor

# ---------------------------- PREPROCESSING ----------------------------

def preprocessing(X, numeric_preprocessor, categorical_preprocessor, is_train=True):
    X = X.copy()
    if is_train:
        X[numerical_feats] = numeric_preprocessor.fit_transform(X[numerical_feats])
        X_cat = categorical_preprocessor.fit_transform(X[categorical_feats]).toarray()
    else:
        X[numerical_feats] = numeric_preprocessor.transform(X[numerical_feats])
        X_cat = categorical_preprocessor.transform(X[categorical_feats]).toarray()

    X_cat = pd.DataFrame(X_cat, columns=categorical_preprocessor.get_feature_names_out())
    X = X.drop(columns=categorical_feats).reset_index(drop=True)
    X = pd.concat([X, X_cat], axis=1)
    return X, numeric_preprocessor, categorical_preprocessor

#### Training function

In [9]:
# def train(trial, df_train: pd.DataFrame, df_val: pd.DataFrame = None, use_pruner: bool = False):
#     X_train,y_train = df_train.drop(columns=['y']), df_train['y']
    
#     if df_val is not None:
#         X_val,y_val = df_val.drop(columns=['y']), df_val['y'] 

#     #Preprocessing
#     numeric_preprocessor = StandardScaler()
#     categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
    
#     X_train,numeric_preprocessor,categorical_preprocessor = preprocessing(X_train,
#                                                                           numeric_preprocessor,
#                                                                           categorical_preprocessor,
#                                                                           is_train=True)
#     if df_val is not None:
#         X_val,_,_ = preprocessing(X_val,
#                                   numeric_preprocessor,categorical_preprocessor,
#                                   is_train=False)

#     #Build model & optimizer
#     model = create_model(trial,X_train.shape[1])
#     optimizer = create_optimizer(trial)
    
#     callbacks = []
#     if use_pruner:
#         callbacks.append(optuna.integration.TFKerasPruningCallback(trial,'val_f1_m'))

#     model.compile(loss='binary_crossentropy',optimizer=optimizer,
#                   metrics=[f1_m],
#                  )
#     history = model.fit(X_train,y_train,
#                         epochs=trial.suggest_int('epoch',15,50),
#                         batch_size=64,
#                         validation_data=(X_val,y_val) if df_val is not None else None,
#                         callbacks=callbacks,
#                         verbose=False
#                        )
#     if df_val is not None:
#         return np.mean(history.history['val_f1_m'])
#     else:
#         return model

# ---------------------------- TRAINING FUNCTION ----------------------------

def train(trial, df_train, df_val=None, use_pruner=False):
    X_train, y_train = df_train.drop(columns=['y']), df_train['y']
    X_val, y_val = None, None

    if df_val is not None:
        X_val, y_val = df_val.drop(columns=['y']), df_val['y']

    # Preprocessing
    numeric_preprocessor = StandardScaler()
    categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
    X_train, numeric_preprocessor, categorical_preprocessor = preprocessing(X_train, numeric_preprocessor, categorical_preprocessor, is_train=True)
    
    if df_val is not None:
        X_val, _, _ = preprocessing(X_val, numeric_preprocessor, categorical_preprocessor, is_train=False)

    model = create_model(trial, X_train.shape[1])
    optimizer = create_optimizer(trial)

    callbacks = []
    if use_pruner and df_val is not None:
        callbacks.append(optuna.integration.TFKerasPruningCallback(trial, 'val_f1_m'))

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[f1_m])
    history = model.fit(
        X_train, y_train,
        epochs=trial.suggest_int('epoch', 15, 50),
        batch_size=64,
        validation_data=(X_val, y_val) if df_val is not None else None,
        callbacks=callbacks,
        verbose=0
    )

    if df_val is not None:
        return np.mean(history.history['val_f1_m'])
    return model


#### The Objective Function

In [10]:
# def objective(trial: optuna.trial.Trial, df_train: pd.DataFrame, use_pruner: bool = False): 
#     #Split into Train and Validation data
#     df_train_hp, df_val = train_test_split(df_train, test_size=0.1, random_state=0)
    
#     # Train and Validate Model
#     val_f1_score = train(trial, df_train_hp, df_val, use_pruner)
        
#     return val_f1_score

# ---------------------------- OBJECTIVE FUNCTION ----------------------------

def objective(trial, df_train, use_pruner=False):
    df_train_hp, df_val = train_test_split(df_train, test_size=0.1, random_state=0)
    val_f1_score = train(trial, df_train_hp, df_val, use_pruner)
    return val_f1_score

#### Final Training & Evaluation Function: `To Test The Best Set of Hyperparmeters`

In [11]:
# def train_and_evaluate_final(df_train: pd.DataFrame, df_test: pd.DataFrame, **kwargs):
#     X_train,y_train = df_train.drop(columns=['y']), df_train['y']
#     X_test,y_test = df_test.drop(columns=['y']), df_test['y'] 
    
#     # Preprocessing
#     numeric_preprocessor = StandardScaler()
#     categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
#     X_train,numeric_preprocessor,categorical_preprocessor = preprocessing(X_train,
#                                                                           numeric_preprocessor,
#                                                                           categorical_preprocessor,
#                                                                           is_train=True)
#     X_test,_,_ = preprocessing(X_test,numeric_preprocessor,categorical_preprocessor,
#                               is_train=False)

#     #Build model
#     input_size = X_train.shape[1]
#     model = Sequential()
#     model.add(Dense(input_size,input_shape=(input_size,),activation='relu')) 

#     num_layers = kwargs.get('num_layers',0)  
#     for layer_i in range(num_layers):  
#         n_units = kwargs.get(f'n_units_layer_{layer_i}',0)  
#         dropout_rate = kwargs.get(f'dropout_rate_layer_{layer_i}',0)  
#         actv_func = kwargs.get(f'actv_func_layer_{layer_i}','relu')  

#         model.add(Dropout(dropout_rate))  
#         model.add(Dense(n_units,activation=actv_func)) 

#     model.add(Dense(1,activation='sigmoid'))
    
#     #Build Optimizer
#     opt_kwargs = {} 
#     opt_selected = kwargs.get('optimizer', 'Adam')
#     if opt_selected == 'SGD': 
#         opt_kwargs['lr'] = kwargs.get('sgd_lr',1e-5) 
#         opt_kwargs['momentum'] = kwargs.get('sgd_momentum',1e-5) 
#     else: #’Adam’ 
#         opt_kwargs['lr'] = kwargs.get('adam_lr',1e-5) 

#     optimizer = getattr(tf.optimizers,opt_selected)(**opt_kwargs) 
    
#     #Training process
#     model.compile(loss='binary_crossentropy',optimizer=optimizer,
#                   metrics=[f1_m],
#                  )
#     print(model.summary())
#     history = model.fit(X_train,y_train,
#                         epochs=kwargs.get('epoch',15),
#                         batch_size=64,
#                         validation_data=None,
#                         verbose=True
#                        )
    
#     # Evaluation Process
#     y_test_pred_probas = model.predict(X_test)
#     y_test_pred = [1 if x[0]>0.5 else 0 for x in y_test_pred_probas]
    
#     print("="*100)
#     print("F1-Score on Test Data: ",f1_score(y_test, y_test_pred))

# ---------------------------- FINAL TRAINING ----------------------------

def train_and_evaluate_final(df_train, df_test, **kwargs):
    X_train, y_train = df_train.drop(columns=['y']), df_train['y']
    X_test, y_test = df_test.drop(columns=['y']), df_test['y']

    numeric_preprocessor = StandardScaler()
    categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")

    X_train, numeric_preprocessor, categorical_preprocessor = preprocessing(X_train, numeric_preprocessor, categorical_preprocessor, is_train=True)
    X_test, _, _ = preprocessing(X_test, numeric_preprocessor, categorical_preprocessor, is_train=False)

    model = Sequential()
    model.add(Dense(X_train.shape[1], input_shape=(X_train.shape[1],), activation='relu'))

    for i in range(kwargs.get('num_layers', 0)):
        model.add(Dropout(kwargs.get(f'dropout_rate_layer_{i}', 0)))
        model.add(Dense(kwargs.get(f'n_units_layer_{i}', 10), activation=kwargs.get(f'actv_func_layer_{i}', 'relu')))

    model.add(Dense(1, activation='sigmoid'))

    opt_name = kwargs.get('optimizer', 'Adam')
    if opt_name == 'SGD':
        optimizer = SGD(
            learning_rate=kwargs.get('sgd_lr', 1e-5),
            momentum=kwargs.get('sgd_momentum', 1e-5)
        )
    else:
        optimizer = Adam(learning_rate=kwargs.get('adam_lr', 1e-5))

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[f1_m])
    model.fit(X_train, y_train, epochs=kwargs.get('epoch', 15), batch_size=64, verbose=1)

    y_test_pred_proba = model.predict(X_test)
    y_test_pred = [1 if x > 0.5 else 0 for x in y_test_pred_proba]

    print("=" * 100)
    print("F1-Score on Test Data:", f1_score(y_test, y_test_pred))

## TPE

In [12]:
df = pd.read_csv("/Users/joaquinromero/Desktop/HPTP/data/train.csv", sep=";")

In [13]:
df['y'] = df['y'].map({'yes':1,'no':0})

### Train/Test Split

In [14]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=0) 

#### Placing Numerical Features

In [15]:
numerical_feats = list(df_train.drop(columns='y').select_dtypes(include=np.number).columns)

#### Placing Categorical Features

In [16]:
categorical_feats = list(df_train.drop(columns='y').select_dtypes(exclude=np.number).columns)

### Performing Hyperparameter Tuning with TPE

In [19]:
# study = optuna.create_study(direction='maximize',
#                             sampler=optuna.samplers.TPESampler(seed=0),
#                            )
# study.optimize(lambda trial: objective(trial, df_train),
#                n_trials=50, n_jobs=-1,
#               )

In [22]:
# print("Best Trial:")
# best_trial = study.best_trial

# print("    Value: ", best_trial.value)

# print("    Hyperparameters: ")
# for key, value in best_trial.params.items():
#     print(f"        {key}: {value}")

In [23]:
# best_trial.params

In [24]:
# train_and_evaluate_final(df_train, df_test, **best_trial.params)